<a href="https://colab.research.google.com/github/HiHippie/Awesome-Visual-Instruction-Tuning/blob/main/llama_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>